In [1]:
import os
import sys
curr_dir=os.getcwd()
p=curr_dir.find('dev')
root=curr_dir[:p]
sys.path.append(root+'lib')
import psi4
import numpy as np
from P4toC4_aux import *

In [2]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", "cc-pVDZ")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("REFERENCE", "UHF")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

# O
# H 1 0.96
# H 1 0.96 2 104.5

h2o = psi4.geometry("""
1 2
O            0.000000000000     0.000000000000    -0.065775570547
H            0.000000000000    -0.759061990794     0.521953018286
H            0.000000000000     0.759061990794     0.521953018286
symmetry c1
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-75.63212241011342

In [3]:
# Print a new basis set in GENBAS format
basisset=wf.basisset()
#print(basisset.genbas())

In [4]:
mapping, scale = basis_mapping(wf.basisset(), verbose=0)
mapping

array([ 0,  1,  2,  7,  3,  5,  8,  4,  6,  9, 11, 13, 12, 10, 14, 15, 18,
       16, 17, 19, 20, 23, 21, 22])

In [5]:
n_a, n_b = wf.nalpha(), wf.nbeta()
print(f'Na={n_a}  Nb={n_b}')
eps_a = np.array(wf.epsilon_a())
eps_b = np.array(wf.epsilon_a())
s=27.2114

for i in range(0,max(n_a,n_b)+2):
    print(f' {i+1:3d}  {eps_a[i]*s:10.4f} {eps_b[i]*s:10.4f}')

Na=5  Nb=4
   1   -575.2463  -575.2463
   2    -52.0426   -52.0426
   3    -33.1132   -33.1132
   4    -30.7389   -30.7389
   5    -29.8502   -29.8502
   6     -3.8686    -3.8686
   7     -1.6020    -1.6020


In [7]:
Ca_C4 = psi4_to_c4(wf.Ca().np, mapping, scale)
Cb_C4 = psi4_to_c4(wf.Cb().np, mapping, scale)

In [25]:
write_oldmos('PSIMOS', Ca_C4, Cbs=Cb_C4)